In [ ]:
%pip install -r requirements.txt

In [1]:
import pickle
import os
from ml_aspade import ml_aspade_train
from train_eval import train_and_save_model, evaluate_model


In [ ]:
win_len = 250
win_shift = int(win_len / 4)

!python training_data_gen_new.py \
    --audio_dir "speech_data" \
    --cnt 200 \
    --train_dir "train_data" \
    --test_dir "test_data" \
    --output_path "pkl_data" \
    --target_fs_values 16000 \
    --clipping_thresholds 0.1 0.2 \
    --time_clip 1 \
    --win_len {win_len} \
    --win_shift {win_shift} \
    --delta 300

In [1]:
import pickle

# Path to your .pkl file
file_path = "/data2/AAG/MTech_Project_Second_Part/pkl_data/training_data.pkl"

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)


# Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from model_new import ComplexDFTAttentionModel, ComplexDFTDataset, train_complex_dft_model, save_model


# Set a fixed random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Create a model with frequency-aware attention
max_sparsity = 500
model = ComplexDFTAttentionModel(dft_size=1000, max_sparsity=max_sparsity)

# Create dummy data for demonstration
batch_size = 512
epochs = 600


import pickle

# Path to your .pkl file
file_path = "/data2/AAG/MTech_Project_Second_Part/pkl_data/training_data.pkl"

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)


num_samples = len(data)

rp_in = []
rp_out = []
ip_in = []
ip_out = []
sp_out = []



for i in range(num_samples):
    rp_in.append(np.real(data[i][0]))
    rp_out.append(np.real(data[i][1]))
    ip_in.append(np.imag(data[i][0]))
    ip_out.append(np.imag(data[i][1]))
    sp_out.append(data[i][2])


# Combine real and imaginary into single vectors
inputs = np.concatenate([rp_in, ip_in], axis=1)
targets_dft = np.concatenate([rp_out, ip_out], axis=1)

# Ensure sparsity values are within range (0 to max_sparsity)
targets_sparsity = sp_out

# # Create dataset and loader with explicit max_sparsity
# train_dataset = ComplexDFTDataset(
#     inputs=inputs,
#     targets_dft=targets_dft,
#     targets_sparsity=targets_sparsity,
#     max_sparsity=max_sparsity
# )
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (e.g., 80% train, 20% val)
train_inputs, val_inputs, train_targets_dft, val_targets_dft, train_sparsity, val_sparsity = train_test_split(
    inputs, targets_dft, targets_sparsity, test_size=0.2, random_state=seed, shuffle=True
)

# Create separate datasets
train_dataset = ComplexDFTDataset(
    inputs=train_inputs,
    targets_dft=train_targets_dft,
    targets_sparsity=train_sparsity,
    max_sparsity=max_sparsity
)

val_dataset = ComplexDFTDataset(
    inputs=val_inputs,
    targets_dft=val_targets_dft,
    targets_sparsity=val_sparsity,
    max_sparsity=max_sparsity
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Train the model with error handling
try:
    # First check if CUDA is available and print a helpful message
    if torch.cuda.is_available():
        print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is not available. Using CPU.")
        
    train_complex_dft_model(model, train_loader, val_loader,  epochs=epochs)
except Exception as e:
    print(f"Training failed with error: {str(e)}")
    print("Try running with CPU only by forcing: device = torch.device('cpu')")

# Save the model
save_model(model, "complex_dft_model.pth")



# New model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

# Import classes from the previously shared code
# Assuming the code is saved as complex_dft_unet.py
# from model_new_new import ComplexDFTUNet, ComplexDFTDataset, train_complex_dft_unet, save_model
from train_eval_new import ComplexDFTDataset, ComplexDFTUNet, debug_model, train_complex_dft_unet, save_model, find_latest_checkpoint

# Set a fixed random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Create a model with frequency-aware attention
max_sparsity = 500
model = ComplexDFTUNet(dft_size=1000, max_sparsity=max_sparsity)  # Using ComplexDFTUNet instead

# Create dummy data for demonstration
batch_size = 512
epochs = 300

# Path to your .pkl file
file_path = "/data2/AAG/MTech_Project_Second_Part/pkl_data/training_data.pkl"

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)

num_samples = len(data)

rp_in = []
rp_out = []
ip_in = []
ip_out = []
sp_out = []

for i in range(num_samples):
    rp_in.append(np.real(data[i][0]))
    rp_out.append(np.real(data[i][1]))
    # Fix: Use np.imag for imaginary parts
    ip_in.append(np.imag(data[i][0]))  
    ip_out.append(np.imag(data[i][1]))
    sp_out.append(data[i][2])

# Combine real and imaginary into single vectors
inputs = np.concatenate([rp_in, ip_in], axis=1)
targets_dft = np.concatenate([rp_out, ip_out], axis=1)

# Ensure sparsity values are within range (0 to max_sparsity)
targets_sparsity = sp_out

from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (e.g., 80% train, 20% val)
train_inputs, val_inputs, train_targets_dft, val_targets_dft, train_sparsity, val_sparsity = train_test_split(
    inputs, targets_dft, targets_sparsity, test_size=0.2, random_state=seed, shuffle=True
)

# Create separate datasets
train_dataset = ComplexDFTDataset(
    inputs=train_inputs,
    targets_dft=train_targets_dft,
    targets_sparsity=train_sparsity,
    max_sparsity=max_sparsity
)

val_dataset = ComplexDFTDataset(
    inputs=val_inputs,
    targets_dft=val_targets_dft,
    targets_sparsity=val_sparsity,
    max_sparsity=max_sparsity
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Optional: Debug the model to check dimensions
debug_model(model, train_loader)

# Set up checkpoint directory
checkpoint_dir = "dft_model_checkpoints"
checkpoint_freq = 5  # Save a checkpoint every 5 epochs

# Train the model with error handling
try:
    # First check if CUDA is available and print a helpful message
    if torch.cuda.is_available():
        print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is not available. Using CPU.")
    
    # Check for existing checkpoints
    latest_checkpoint = find_latest_checkpoint(checkpoint_dir)
    
    if latest_checkpoint:
        print(f"Found checkpoint at {latest_checkpoint}. Resuming training...")
        resume_from = latest_checkpoint
    else:
        print("No checkpoint found. Starting training from scratch.")
        resume_from = None
    
    # Train the model with checkpointing
    loss_history = train_complex_dft_unet(
        model=model, 
        train_loader=train_loader, 
        val_loader=val_loader,
        epochs=epochs,
        plot_loss=True,  # Enable automatic plotting
        checkpoint_dir=checkpoint_dir,
        checkpoint_freq=checkpoint_freq,
        resume_from=resume_from
    )

    # You can access individual loss histories from the returned dictionary
    train_losses = loss_history['train_loss']
    dft_losses = loss_history['dft_loss']
    sparsity_losses = loss_history['sparsity_loss']
    
    # Create additional plots if needed
    # For example, plot the ratio of DFT loss to sparsity loss over time
    plt.figure(figsize=(10, 6))
    loss_ratio = [d/s if s > 0 else d for d, s in zip(dft_losses, sparsity_losses)]
    plt.plot(range(1, epochs+1), loss_ratio, 'b-')
    plt.title('Ratio of DFT Loss to Sparsity Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss Ratio')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(checkpoint_dir, "loss_ratio.png"), dpi=300)
    plt.show()

except Exception as e:
    print(f"Training failed with error: {str(e)}")
    print("Try running with CPU only by forcing: device = torch.device('cpu')")

# Save the final model
final_model_path = os.path.join(checkpoint_dir, "complex_dft_unet_final_model.pth")
save_model(model, final_model_path)
print(f"Final model saved to {final_model_path}")

Running debug mode to check dimensions...
Input shape: torch.Size([512, 2000])
Reshaped input: torch.Size([512, 1, 2000])
After inc: torch.Size([512, 64, 2000])
After down1: torch.Size([512, 128, 1000]), skip1: torch.Size([512, 128, 2000])
After down2: torch.Size([512, 256, 500]), skip2: torch.Size([512, 256, 1000])


KeyboardInterrupt: 

# Evaluation

In [2]:
import pickle
import os
from ml_aspade import ml_aspade_train
from train_eval import train_and_save_model, evaluate_model
from model_new import ComplexDFTAttentionModel, load_model
from train_eval_new import ComplexDFTUNet
import torch

sparsity_model_path = "complex_dft_unet_model.pth"
test_audio_dir = "custom_sound"
output_dir = "/data2/AAG/MTech_Project_Second_Part/test_data_output"

target_fs_values = [16000]
clipping_thresholds = [0.2]
time_clip = [1]
max_sparsity = 1000
k_classes=100



loaded_model = load_model(ComplexDFTUNet, sparsity_model_path)



# 1. Set device explicitly to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_model = loaded_model.to(device)

# Run evaluation
results_df, summary = evaluate_model(
    test_audio_dir=test_audio_dir,
    output_dir=output_dir,
    target_fs_values=target_fs_values,
    clipping_thresholds=clipping_thresholds,
    time_clip=time_clip,
    model_path=sparsity_model_path,
    loaded_model=loaded_model,
    device=device)

# Print summary statistics
print("\nEvaluation Summary:")
print(summary)


Processing files:   0%|          | 0/1 [00:00<?, ?it/s]


Processing: 6930-81414-0024.wav (fs=16000, threshold=0.2, duration=1)
Generating clipped signal...


[[-0.07560468 -0.07555125 -0.07982679 ... -0.07825848 -0.07736034
  -0.07407521]]
148
[[-0.07567772 -0.07448196 -0.07842342 ... -0.07744353 -0.07548506
  -0.07516748]]
148
[[-0.07573517 -0.07397032 -0.07793611 ... -0.07739313 -0.0756295
  -0.07470691]]
148
[[-0.07534762 -0.07425734 -0.07770101 ... -0.07503348 -0.07823052
  -0.07482532]]
148
[[-0.07530093 -0.07444736 -0.0775483  ... -0.07657904 -0.07679679
  -0.07461952]]
148
[[-0.07504307 -0.07536486 -0.07640035 ... -0.07563413 -0.07668316
  -0.0741298 ]]
148
[[-0.07528232 -0.07578619 -0.07939675 ... -0.0786192  -0.07798985
  -0.07405207]]
148
[[-0.07572125 -0.07514391 -0.08024652 ... -0.07836597 -0.07553323
  -0.07501822]]
148
[[-0.07538454 -0.07437316 -0.07865262 ... -0.07824518 -0.07516307
  -0.07492942]]
148
[[-0.07587842 -0.07435446 -0.07827526 ... -0.07766348 -0.07565081
  -0.07519371]]
148


[[-0.07539479 -0.07484386 -0.07743374 ... -0.07642864 -0.07817011
  -0.07503872]]
148
[[-0.07527731 -0.07465743 -0.07689767 ... -0.07616401 -0.07632628
  -0.0744746 ]]
148
[[-0.07520255 -0.07482909 -0.07680343 ... -0.07640134 -0.07619257
  -0.07410258]]
148
[[-0.07554822 -0.0740929  -0.07773724 ... -0.07694258 -0.07592145
  -0.07449695]]
148
[[-0.07535139 -0.07463022 -0.07674228 ... -0.07645264 -0.07641677
  -0.07430862]]
148
[[-0.07539814 -0.07443855 -0.07785612 ... -0.07667837 -0.0763153
  -0.07432749]]
148
[[-0.07542598 -0.07440257 -0.07725473 ... -0.07659864 -0.07621178
  -0.07433792]]
148
[[-0.07544722 -0.07437271 -0.07763284 ... -0.07671969 -0.07605846
  -0.0743463 ]]
148
[[-0.07536332 -0.07471156 -0.07688772 ... -0.07615954 -0.07637188
  -0.07425229]]
148
[[-0.07556377 -0.07507913 -0.07877918 ... -0.07779814 -0.07686041
  -0.07412551]]
148


[[-0.07561132 -0.07442296 -0.07805858 ... -0.0773794  -0.07569174
  -0.07485103]]
148
[[-0.07542473 -0.07431495 -0.07783074 ... -0.07715219 -0.07606235
  -0.07420076]]
148
[[-0.07591099 -0.07404174 -0.07775973 ... -0.07741584 -0.07580543
  -0.07486187]]
148
[[-0.07531638 -0.07446292 -0.07756033 ... -0.07625809 -0.0779644
  -0.07465369]]
148
[[-0.07555223 -0.07457716 -0.07832281 ... -0.07529965 -0.07805169
  -0.07454731]]
148
[[-0.07528665 -0.07453433 -0.07733897 ... -0.07680196 -0.0785625
  -0.07474926]]
148
[[-0.07503106 -0.07520357 -0.07693852 ... -0.07570486 -0.07647005
  -0.07425149]]
148
[[-0.07502246 -0.07559889 -0.07718819 ... -0.07869121 -0.07749121
  -0.07397211]]
148
[[-0.07622772 -0.07688181 -0.08102867 ... -0.07903957 -0.07625214
  -0.07506236]]
148
[[-0.07530902 -0.07541347 -0.07978848 ... -0.07833029 -0.07602476
  -0.07575919]]
148


...

[[-0.0754602  -0.07745591 -0.07894683 ... -0.08079692 -0.07121755
  -0.07859171]]
148
[[-0.07564501 -0.07381348 -0.08094509 ... -0.07841747 -0.0738384
  -0.07670529]]
148
[[-0.07571162 -0.074792   -0.08057416 ... -0.0747534  -0.08293162
  -0.0751572 ]]
148
[[-0.07516845 -0.07537136 -0.07634823 ... -0.07568265 -0.0797929
  -0.07478714]]
148
[[-0.07502609 -0.07576971 -0.07669567 ... -0.07518719 -0.08038778
  -0.07468878]]
148
[[-0.07548632 -0.07448961 -0.07694254 ... -0.07610376 -0.07679957
  -0.07433779]]
148
[[-0.07522227 -0.07604578 -0.07863122 ... -0.08049867 -0.07830834
  -0.07397279]]
148
[[-0.07553247 -0.07738937 -0.08155812 ... -0.08142515 -0.07756084
  -0.07539738]]
148
[[-0.07576575 -0.07678639 -0.08034858 ... -0.08011138 -0.07380227
  -0.07743943]]
148
[[-0.07534657 -0.07441719 -0.08088904 ... -0.07955649 -0.07377274
  -0.076901  ]]
148
[[-0.07699627 -0.07488357 -0.08236139 ... -0.07659941 -0.07986782
  -0.07604378]]
148


[[-0.07523397 -0.07537659 -0.07763954 ... -0.07632373 -0.08215146
  -0.07395572]]
148
[[-0.07473014 -0.07574459 -0.07562667 ... -0.07420883 -0.0808573
  -0.07494739]]
148
[[-0.07534185 -0.07409183 -0.07648353 ... -0.07710911 -0.07776622
  -0.07439753]]
148
[[-0.07486912 -0.07532747 -0.07677764 ... -0.07570288 -0.07706978
  -0.07343528]]
148
[[-0.0753005  -0.0764865  -0.08121406 ... -0.08061679 -0.07888812
  -0.07437138]]
148
[[-0.07606283 -0.07619869 -0.08122516 ... -0.07937986 -0.07610806
  -0.07623278]]
148
[[-0.07556019 -0.07704914 -0.08096451 ... -0.08016453 -0.07697151
  -0.07637175]]
148
[[-0.0751752  -0.07425272 -0.08114483 ... -0.07831386 -0.07379514
  -0.07590171]]
148
[[-0.0758351  -0.07462575 -0.07995047 ... -0.07569537 -0.08109656
  -0.07519282]]
148
[[-0.07508836 -0.07588774 -0.07742929 ... -0.07494777 -0.0800987
  -0.07445571]]
148
[[-0.07537522 -0.07437675 -0.07661612 ... -0.07658892 -0.07860257
  -0.07461123]]
148


Processing: 100%|██████████| 131/131 [00:03<00:00, 39.22iteration/s]


[[-0.07556216 -0.07540046 -0.07902101 ... -0.07793518 -0.07719414
  -0.07387175]]
148
[[-0.07540353 -0.07426181 -0.07730407 ... -0.07679249 -0.07586318
  -0.07448933]]
148
[[-0.07519783 -0.07499876 -0.07673815 ... -0.07590375 -0.07649934
  -0.07411554]]
148


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

# Git Push

In [1]:
!cd /data2/AAG/MTech_Project_Second_Part

In [2]:
!git init

Initialized empty Git repository in /data2/AAG/MTech_Project_Second_Part/.git/


In [8]:
!echo -e "*.wav\n*.pkl\n*.pth\n*.log\naagproj/\n__pycache__/\n*.pyc\n.env\n.DS_Store" > .gitignore

In [9]:
!cat .gitignore

*.wav
*.pkl
*.pth
*.log
aagproj/
__pycache__/
*.pyc
.env
.DS_Store


In [11]:
!git reset
!git add .
!git commit -m "Initial backup (excluding .wav files)"
!git remote add origin https://github.com/AAG1201/MTech_Project_Second_Part.git
!git branch -M main
!git push -u origin main

Unstaged changes after reset:
M	main_ml.ipynb
[main c71d82e] Initial backup (excluding .wav files)
 1 file changed, 36 insertions(+), 1 deletion(-)
fatal: remote origin already exists.
Username for 'https://github.com': 